# Test Selection Analysis

In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance

In [86]:
def load_data(_train, _test, fname) :
    fpath = "trained_models/{}/eval/{}/{}.txt".format(_train, _test, fname)
    return np.loadtxt(fpath).astype(np.int)

train = "autoattack"
test = "pgd"

y_original = load_data(train, test, "Y_original")
y_original_pred = load_data(train, test, "Y_original_pred")
y_adv = load_data(train, test, "Y_adv")
y_adv_pred = load_data(train, test, "Y_adv_pred")

In [87]:
def get_robust_data(train, test):
    y_adv = load_data(train, test, "Y_adv")
    y_adv_pred = load_data(train, test, "Y_adv_pred")
    return y_adv, y_adv_pred

train = "pgd"
test = "autoattack"
y_adv, y_adv_pred = get_robust_data(train, test)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(pgd,autoattack): [3 8 8 ... 5 1 7]
Y_adv_pred(pgd,autoattack): [5 8 8 ... 5 1 7]


In [88]:
train = "fgsm"
test = "autopgd"
y_adv, y_adv_pred = get_robust_data(train, test)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(fgsm,autopgd): [3 8 8 ... 5 1 7]
Y_adv_pred(fgsm,autopgd): [3 8 8 ... 5 1 7]


In [89]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack", "pixelattack", "spatialtransformation"]

In [90]:
train = attacks.copy()
train.append("all")
train.append("combine/autopgd_pixelattack_spatialtransformation_squareattack")

test = attacks.copy()
test.append("all")

In [91]:
predictions = {}
labels = {}

for a1 in train :
    p = np.array([])
    l = np.array([])
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        p = np.concatenate((p, y_pred))
        l = np.concatenate((l, y))
    predictions[a1] = {}
    labels[a1] = {}
    predictions[a1]["all"] = p
    labels[a1]["all"] = l

for a1 in train :
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        predictions[a1][a2] = y_pred
        labels[a1][a2] = y

In [92]:
# predictions

In [93]:
# labels

In [94]:
accs = {}
for tr in train :
    accs[tr] = {}
    for ts in test :
        accs[tr][ts] = (labels[tr][ts] == predictions[tr][ts]).sum().item() / len(labels[tr][ts])

    
accs = pd.DataFrame(data=accs).T
accs

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
autoattack,0.7885,0.7864,0.7829,0.7922,0.7510,0.7786,0.7787,0.7902,0.7842,0.1209,0.4102,0.687618
autopgd,0.7839,0.7838,0.7767,0.7899,0.7508,0.7753,0.7783,0.7849,0.7787,0.1322,0.3450,0.679955
bim,0.7839,0.7816,0.7838,0.7902,0.7548,0.7824,0.7808,0.7902,0.7839,0.1190,0.3833,0.684900
cw,0.7891,0.7886,0.7871,0.8083,0.7518,0.7862,0.7929,0.8034,0.7999,0.1140,0.3890,0.691845
fgsm,0.7774,0.7756,0.7768,0.7856,0.7649,0.7754,0.7792,0.7827,0.7777,0.1583,0.3599,0.683045
pgd,0.7758,0.7793,0.7753,0.7816,0.7515,0.7751,0.7746,0.7813,0.7770,0.1379,0.3621,0.679227
deepfool,0.7825,0.7805,0.7758,0.7958,0.7608,0.7727,0.7910,0.7934,0.7849,0.0972,0.3898,0.684036
newtonfool,0.7819,0.7842,0.7793,0.8015,0.7500,0.7768,0.7892,0.7997,0.7927,0.1056,0.3822,0.685736
squareattack,0.7343,0.7369,0.7338,0.7971,0.6910,0.7264,0.7544,0.7887,0.8299,0.1035,0.3833,0.661755
pixelattack,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.7986,0.1000,0.163509


In [95]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
all,0.8146,0.8154,0.8137,0.8306,0.7792,0.8122,0.8237,0.8274,0.8263,0.1443,0.7084,0.745073
combine/autopgd_pixelattack_spatialtransformation_squareattack,0.7994,0.7993,0.7891,0.8252,0.7435,0.7880,0.8032,0.8217,0.8320,0.1371,0.7676,0.736918
cw,0.7891,0.7886,0.7871,0.8083,0.7518,0.7862,0.7929,0.8034,0.7999,0.1140,0.3890,0.691845
autoattack,0.7885,0.7864,0.7829,0.7922,0.7510,0.7786,0.7787,0.7902,0.7842,0.1209,0.4102,0.687618
newtonfool,0.7819,0.7842,0.7793,0.8015,0.7500,0.7768,0.7892,0.7997,0.7927,0.1056,0.3822,0.685736
bim,0.7839,0.7816,0.7838,0.7902,0.7548,0.7824,0.7808,0.7902,0.7839,0.1190,0.3833,0.684900
deepfool,0.7825,0.7805,0.7758,0.7958,0.7608,0.7727,0.7910,0.7934,0.7849,0.0972,0.3898,0.684036
fgsm,0.7774,0.7756,0.7768,0.7856,0.7649,0.7754,0.7792,0.7827,0.7777,0.1583,0.3599,0.683045
autopgd,0.7839,0.7838,0.7767,0.7899,0.7508,0.7753,0.7783,0.7849,0.7787,0.1322,0.3450,0.679955
pgd,0.7758,0.7793,0.7753,0.7816,0.7515,0.7751,0.7746,0.7813,0.7770,0.1379,0.3621,0.679227


## Time Execution Measurement

In [96]:
import datetime

def get_adv_train_time_execution(train) :
    fpath = "trained_models/{}/output.log".format(train)
    
    file = open(fpath)
    lines = file.readlines()
    file.close()
    
    start_time = None
    end_time = None
    
    datefmt='%Y/%m/%d %H:%M:%S'
    
    
    for l in lines :
        token = l.split("-")
        date_str = token[0][1:-2]
        date_time = datetime.datetime.strptime(date_str, datefmt)
        if token[1][1:18] == "Namespace(attack=" :
            start_time = date_time
        else :
            end_time = date_time
    
    if end_time > start_time :
        return end_time-start_time
    raise ValueError("Duration not found")
    
_train = "autoattack"
_train = "all"
get_adv_train_time_execution(_train) + get_adv_train_time_execution("fgsm") 

datetime.timedelta(seconds=41410)

In [108]:
durations = []
for tr in train :
    if tr[:7] == "combine" :
        duration = None
#         duration = get_adv_train_time_execution(tr)
        for i in range(len(attacks)) :
            a = attacks[i]
            if i == 0 :
                duration = get_adv_train_time_execution(a)
            else :
                duration += get_adv_train_time_execution(a)
#         duration+= get_adv_train_time_execution(tr)

        durations.append(duration)
    else :
        durations.append(get_adv_train_time_execution(tr))

accs["duration"] = durations

In [109]:
accs

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
autoattack,0.7885,0.7864,0.7829,0.7922,0.7510,0.7786,0.7787,0.7902,0.7842,0.1209,0.4102,0.687618,0 days 01:59:04
autopgd,0.7839,0.7838,0.7767,0.7899,0.7508,0.7753,0.7783,0.7849,0.7787,0.1322,0.3450,0.679955,0 days 01:44:51
bim,0.7839,0.7816,0.7838,0.7902,0.7548,0.7824,0.7808,0.7902,0.7839,0.1190,0.3833,0.684900,0 days 01:44:02
cw,0.7891,0.7886,0.7871,0.8083,0.7518,0.7862,0.7929,0.8034,0.7999,0.1140,0.3890,0.691845,0 days 01:44:05
fgsm,0.7774,0.7756,0.7768,0.7856,0.7649,0.7754,0.7792,0.7827,0.7777,0.1583,0.3599,0.683045,0 days 00:53:33
pgd,0.7758,0.7793,0.7753,0.7816,0.7515,0.7751,0.7746,0.7813,0.7770,0.1379,0.3621,0.679227,0 days 01:44:21
deepfool,0.7825,0.7805,0.7758,0.7958,0.7608,0.7727,0.7910,0.7934,0.7849,0.0972,0.3898,0.684036,0 days 01:44:35
newtonfool,0.7819,0.7842,0.7793,0.8015,0.7500,0.7768,0.7892,0.7997,0.7927,0.1056,0.3822,0.685736,0 days 01:59:15
squareattack,0.7343,0.7369,0.7338,0.7971,0.6910,0.7264,0.7544,0.7887,0.8299,0.1035,0.3833,0.661755,0 days 01:44:32
pixelattack,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.7986,0.1000,0.163509,0 days 01:43:49


In [103]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
all,0.8146,0.8154,0.8137,0.8306,0.7792,0.8122,0.8237,0.8274,0.8263,0.1443,0.7084,0.745073,0 days 10:36:37
combine/autopgd_pixelattack_spatialtransformation_squareattack,0.7994,0.7993,0.7891,0.8252,0.7435,0.7880,0.8032,0.8217,0.8320,0.1371,0.7676,0.736918,0 days 23:01:08
cw,0.7891,0.7886,0.7871,0.8083,0.7518,0.7862,0.7929,0.8034,0.7999,0.1140,0.3890,0.691845,0 days 01:44:05
autoattack,0.7885,0.7864,0.7829,0.7922,0.7510,0.7786,0.7787,0.7902,0.7842,0.1209,0.4102,0.687618,0 days 01:59:04
newtonfool,0.7819,0.7842,0.7793,0.8015,0.7500,0.7768,0.7892,0.7997,0.7927,0.1056,0.3822,0.685736,0 days 01:59:15
bim,0.7839,0.7816,0.7838,0.7902,0.7548,0.7824,0.7808,0.7902,0.7839,0.1190,0.3833,0.684900,0 days 01:44:02
deepfool,0.7825,0.7805,0.7758,0.7958,0.7608,0.7727,0.7910,0.7934,0.7849,0.0972,0.3898,0.684036,0 days 01:44:35
fgsm,0.7774,0.7756,0.7768,0.7856,0.7649,0.7754,0.7792,0.7827,0.7777,0.1583,0.3599,0.683045,0 days 00:53:33
autopgd,0.7839,0.7838,0.7767,0.7899,0.7508,0.7753,0.7783,0.7849,0.7787,0.1322,0.3450,0.679955,0 days 01:44:51
pgd,0.7758,0.7793,0.7753,0.7816,0.7515,0.7751,0.7746,0.7813,0.7770,0.1379,0.3621,0.679227,0 days 01:44:21


In [104]:
selected_sorted_acc = sorted_acc[:][["all", "duration"]]
selected_sorted_acc

,all,duration
all,0.745073,0 days 10:36:37
combine/autopgd_pixelattack_spatialtransformation_squareattack,0.736918,0 days 23:01:08
cw,0.691845,0 days 01:44:05
autoattack,0.687618,0 days 01:59:04
newtonfool,0.685736,0 days 01:59:15
bim,0.684900,0 days 01:44:02
deepfool,0.684036,0 days 01:44:35
fgsm,0.683045,0 days 00:53:33
autopgd,0.679955,0 days 01:44:51
pgd,0.679227,0 days 01:44:21
